Synthetic Data Generation: We created a synthetic dataset for patient visits.
PostgreSQL Setup: We set up a PostgreSQL database and table.
OpenAI Integration: We integrated a module to translate user queries into SQL and interact with the database.

Generate Synthetic Time Series Dataset
synthetic dataset that simulates patient visits over a year, excluding weekends and public holidays.

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Function to generate synthetic dataset
def generate_synthetic_data(start_date, end_date):
    # Generate all dates between start_date and end_date
    all_dates = pd.date_range(start_date, end_date, freq='B')  # 'B' for business days
    public_holidays = [
        datetime(2023, 1, 1),  # New Year's Day
        datetime(2023, 7, 4),  # Independence Day
        datetime(2023, 12, 25),  # Christmas
    ]
    valid_dates = [date for date in all_dates if date not in public_holidays]

    # Generate random patient data
    np.random.seed(42)  # For reproducibility
    data = {
        'Timestamp': valid_dates,
        'Patient Name': [f'Patient_{i}' for i in range(len(valid_dates))],
        'Sex': np.random.choice(['Male', 'Female'], size=len(valid_dates)),
        'Age': np.random.randint(1, 90, size=len(valid_dates)),
        'Weight': np.random.randint(50, 100, size=len(valid_dates)),  # Weight in kg
        'BloodGroup': np.random.choice(['A+', 'A-', 'B+', 'B-', 'O+', 'O-', 'AB+', 'AB-'], size=len(valid_dates))
    }

    return pd.DataFrame(data)

# Generate data for the year 2023
synthetic_data = generate_synthetic_data('2023-01-01', '2023-12-31')
print(synthetic_data.head())


Geneate Data Using OPenAI module

In [ ]:
import os
import openai
import pandas as pd
from dotenv import load_dotenv
from datetime import datetime, timedelta
import numpy as np

# Load environment variables
load_dotenv()

# Configure OpenAI
openai.api_key = os.getenv("OPENAI_API_KEY")

# Function to generate synthetic patient data using OpenAI
def generate_synthetic_patient_data(num_records):
    patient_data = []
    for _ in range(num_records):
        prompt = (
            "Generate a synthetic patient record with the following fields:\n"
            "1. Timestamp (date of visit)\n"
            "2. Patient Name (first and last name)\n"
            "3. Sex (Male/Female)\n"
            "4. Age (integer between 1 and 90)\n"
            "5. Weight (integer in kg between 50 and 100)\n"
            "6. BloodGroup (one of: A+, A-, B+, B-, O+, O-, AB+, AB-)\n"
            "Provide the data in a CSV format."
        )
        
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=150
        )

        patient_record = response.choices[0].message['content']
        patient_data.append(eval(patient_record))  # Convert JSON string to Python dict

    return pd.DataFrame(patient_data)

# Function to filter valid business days and public holidays
def filter_valid_dates(df):
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    public_holidays = [
        datetime(2023, 1, 1),
        datetime(2023, 7, 4),
        datetime(2023, 12, 25),
    ]
    df = df[~df['Timestamp'].dt.date.isin([holiday.date() for holiday in public_holidays])]
    df = df[df['Timestamp'].dt.weekday < 5]  # Keep only weekdays (Mon-Fri)
    return df

# Generate synthetic patient data for 100 records
num_records = 1000
synthetic_data = generate_synthetic_patient_data(num_records)
synthetic_data = filter_valid_dates(synthetic_data)

print(synthetic_data.head())


Insertion of Time Series Synthetic Data into PostgresDB

In [ ]:
import os
import psycopg2
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Function to insert data into PostgreSQL
def insert_data_to_db(df):
    try:
        connection = psycopg2.connect(
            dbname=os.getenv("DB_NAME"),
            user=os.getenv("DB_USER"),
            password=os.getenv("DB_PASSWORD"),
            host=os.getenv("DB_HOST"),
            port=os.getenv("DB_PORT")
        )
        cursor = connection.cursor()

        insert_query = """
        INSERT INTO patient_data (Timestamp, Patient_Name, Sex, Age, Weight, BloodGroup)
        VALUES (%s, %s, %s, %s, %s, %s)
        """
        for _, row in df.iterrows():
            cursor.execute(insert_query, (row['Timestamp'], row['Patient Name'], row['Sex'], row['Age'], row['Weight'], row['BloodGroup']))

        connection.commit()
    except Exception as e:
        print(f"Error inserting data: {e}")
    finally:
        cursor.close()
        connection.close()

# Insert synthetic data into PostgreSQL
insert_data_to_db(synthetic_data)


Integration of OpenAI Module
interface to interact with the PostgreSQL database using OpenAI. The user will be able to ask questions in plain English, and we will process the queries accordingly.

In [ ]:
import openai

# Configure OpenAI
openai.api_key = os.getenv("OPENAI_API_KEY")

def fetch_data_from_openai(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=150
    )
    return response.choices[0].message['content']

# Function to query the database based on OpenAI output
def query_database(query):
    try:
        connection = psycopg2.connect(
            dbname=os.getenv("DB_NAME"),
            user=os.getenv("DB_USER"),
            password=os.getenv("DB_PASSWORD"),
            host=os.getenv("DB_HOST"),
            port=os.getenv("DB_PORT")
        )
        cursor = connection.cursor()
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Exception as e:
        print(f"Error querying database: {e}")
    finally:
        cursor.close()
        connection.close()

# Function to process user queries
def process_query(user_query):
    prompt = f"Translate the following question to SQL: {user_query}"
    sql_query = fetch_data_from_openai(prompt)
    print("Generated SQL Query:", sql_query)
    result = query_database(sql_query)
    return result

# Example of interacting with the user
def main():
    user_query = input("Ask a question about patient data: ")
    result = process_query(user_query)
    print("Query Result:", result)

if __name__ == "__main__":
    main()
